In [2]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

Cloning into 'TACL2015'...
remote: Enumerating objects: 2294, done.
remote: Counting objects: 100% (2294/2294), done.
remote: Compressing objects: 100% (2234/2234), done.
remote: Total 2294 (delta 203), reused 2103 (delta 55), pack-reused 0
Receiving objects: 100% (2294/2294), 4.51 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (203/203), done.
Cloning into 'nlu-asdiv-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 30 (delta 6), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), 425.56 KiB | 1.67 MiB/s, done.
Cloning into 'grade-school-math'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 14), reused 30 (delta 11), pack-reused 0
Unpacking objects: 100% (36/36), 3.01 MiB | 4.94 MiB/s, done.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored
import wandb

In [2]:
def set_all_seeds():
    """Set all seeds"""
    torch.manual_seed(0)
    np.random.seed(8)
    os.environ['PYTHONHASHSEED']=str(0)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False


In [3]:
import dataset_handler as dh
import helper_func as hf

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

In [11]:
"""Load gsm8k"""
#priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt_codegen.txt" # for codegen
priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt.txt" # for gpt-j
current_dataset = dh.init_dataset_from_name("gsm8k", primingtext_path = priming_text_path)

In [5]:
"""Load asdiv"""
#priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt" # for codegen
priming_text_path = "data/priming_texts/asdiv/asdiv_prefix.txt" # for gpt-j
current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

In [12]:
set_all_seeds()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(100)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Write a program that prints the answer to the following question. Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?
80


In [6]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
#model_args.model = "codegen-350M-mono"
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 467.01s
loading tokenizer
loading tokenizer took 4.00s


In [7]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [8]:
# Set up for CodeGen
config = hf.codegen_gen_args()
config.num_return_sequences = 5 # 4 for gsm8k 5 for asdiv
config.k = 3
config.max_lenght_after_input = 250
config.top_p = 0.95
config.top_k = 50
config.temperature = 0.7
config.min_length = 3

set_all_seeds()
hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

In [13]:
# Set up for gpt-j
config = hf.gptj_gen_args()

torch.manual_seed(42) # very relevant seed for gpt-j
np.random.seed(42) # very relevant seed for gpt-j 
#hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

In [14]:
with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name="@100-gsm8k-gpt_j"):

        #set_all_seeds()
        torch.manual_seed(42) # very relevant seed for gpt-j
        np.random.seed(42) # very relevant seed for gpt-j
        pass_at_k = hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

        wandb.log({"pass_at_k": pass_at_k})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TESTING STARTED
@sample 5 -> Pass@3 = 0.15
@sample 10 -> Pass@3 = 0.15
@sample 15 -> Pass@3 = 0.1
@sample 20 -> Pass@3 = 0.075
@sample 25 -> Pass@3 = 0.06
@sample 30 -> Pass@3 = 0.05
@sample 35 -> Pass@3 = 0.06428571428571428
@sample 40 -> Pass@3 = 0.05625
@sample 45 -> Pass@3 = 0.06666666666666667
@sample 50 -> Pass@3 = 0.06
@sample 55 -> Pass@3 = 0.07272727272727272
@sample 60 -> Pass@3 = 0.06666666666666667
@sample 65 -> Pass@3 = 0.06153846153846154
@sample 70 -> Pass@3 = 0.06785714285714285
@sample 75 -> Pass@3 = 0.07666666666666666
@sample 80 -> Pass@3 = 0.071875
@sample 85 -> Pass@3 = 0.06764705882352941
@sample 90 -> Pass@3 = 0.07222222222222222
@sample 95 -> Pass@3 = 0.06842105263157895
@sample 100 -> Pass@3 = 0.065


Pass@3 = 0.065



pass_at_k,▁
pass_at_k,0.065
